## Convert annotation into um and generate downsampled txt file from resampled file

Generating pandas data frame from vaa3d annotation.eswc file

1. convert them into real units (ie. um) based acquisition microns/pixels
2. Resample this eswc file in V3d (plugin- neuron untilities-resample swc) in a fixes stepsize (ie.1 um)
3. Load the resampled file and downsample them to the corresponding dimensions (ie.25um)
4. saves as transformix compatible format (in case for sample2ara)

__Note that step 2 has to be done in V3d__

__Important__: must have D (case sensitive) in the file name and file path __only__ if it is a dendritic annotation!!!

In progress: get endings based on the parent name


In [2]:
import os

import pandas as pd

import numpy as np

import tkinter as tk
import tkinter.filedialog as fdialog
from tkinter import simpledialog

In [4]:
xy = simpledialog.askfloat("Input", "What is the x and y resolution in um?",
                               minvalue=0.0, maxvalue=100)
z = simpledialog.askfloat("Input", "What is the z resolution in um?",
                               minvalue=0.0, maxvalue=100)
outdir = fdialog.askdirectory(title='Please select the output directory')
anno_file=fdialog.askopenfile(initialdir='Z:\\', title='Select the eswc file containing the annotations').name 

In [6]:
anno=open(anno_file,'r')
anno_data=anno.readlines()
# heading is stored in anno_data[2], 1st line basically useless

headings=anno_data[2].rstrip('\n').split(' ')
annotations=[lines.rstrip('\n').split(' ') for lines in anno_data[3:]]

In [7]:
annotation_df=pd.DataFrame(annotations, columns=headings)


Index(['#n', 'type', 'x', 'y', 'z', 'radius', 'parent', 'seg_id', 'level',
       'mode', 'timestamp', 'TFresindex'],
      dtype='object')

In [8]:
#annotation_df.head()

,#n,type,x,y,z,radius,parent,seg_id,level,mode,timestamp,TFresindex
0,984456,3,4376.667,3841.667,717.111,1.000,984457,0,1,2088763392,1,1
1,984455,3,4377.000,3841.000,717.000,1.000,984456,0,1,2088763392,1,1
2,984457,3,4375.556,3842.667,717.889,1.000,984458,0,1,2088763392,1,1
3,984458,3,4373.444,3843.111,718.333,1.000,984459,0,1,2088763392,1,1
4,984459,3,4373.111,3843.444,718.667,1.000,984460,0,1,2088763392,1,1


In [9]:
annotation_df['x']=pd.to_numeric(annotation_df['x'])*xy
annotation_df['y']=pd.to_numeric(annotation_df['y'])*xy
annotation_df['z']=pd.to_numeric(annotation_df['z'])*z
#annotation_df.head()

,#n,type,x,y,z,radius,parent,seg_id,level,mode,timestamp,TFresindex
0,984456,3,3501.3336,3073.3336,3585.555,1.000,984457,0,1,2088763392,1,1
1,984455,3,3501.6000,3072.8000,3585.000,1.000,984456,0,1,2088763392,1,1
2,984457,3,3500.4448,3074.1336,3589.445,1.000,984458,0,1,2088763392,1,1
3,984458,3,3498.7552,3074.4888,3591.665,1.000,984459,0,1,2088763392,1,1
4,984459,3,3498.4888,3074.7552,3593.335,1.000,984460,0,1,2088763392,1,1


In [10]:
#Write the converted value back to a eswc file to feed into v3d resample_swc

#In this text file, the x y z represents the coordinates in um

#write the first 3 lines exactly like the loaded file
tfile = open(anno_file+'converted.eswc', 'a')
tfile.write(anno_data[0])
tfile.write(anno_data[1])
tfile.write(anno_data[2])
tfile.write(annotation_df.to_string(header=False, index=False))
tfile.close()

## note this file generates more spaces as separators but thankfully resample_Swc plugin didn't care


## Now, take the resampled eswc file and downsample into 25um

Need to remember the resample stepsize (=1 um in the usual case)


In [14]:
outdir = fdialog.askdirectory(title='Please select the output directory')
resample_file=fdialog.askopenfile(initialdir=outdir, title='Select the eswc file containing the converted and recampled annotations').name 

resampled_xyz = simpledialog.askfloat("Input", "What is the x, y and z resolution in um after converting and resampling?",
                               minvalue=0.0, maxvalue=100)
goal_xyz = simpledialog.askfloat("Input", "What do you want to downsample the resolution to '(in um)' ?",
                               minvalue=10, maxvalue=100)

AttributeError: 'NoneType' object has no attribute 'name'

In [4]:
ratioxyz=goal_xyz/resampled_xyz
message = (f"Resampled annotation step size is {resampled_xyz} um in x y z. "
f"downsampling to {goal_xyz} um. "
f"dowmsample ratio is {ratioxyz}.")

message

'Resampled annotation step size is 1.0 um in x y z. downsampling to 25.0 um. dowmsample ratio is 25.0.'

In [5]:
resampled_anno=open(resample_file,'r')
resampled_anno_data=resampled_anno.readlines()
# heading is stored in anno_data[2], 1st line basically useless

headings=resampled_anno_data[2].rstrip('\n').replace(' ', '').split(',')
resampled_annotations=[lines.rstrip('0 1\n').split(' ') for lines in resampled_anno_data[3:]]
#slight modification on replacing and stripping due to the format of the resampled swc

resampled_annotation_df=pd.DataFrame(resampled_annotations, columns=headings)

resampled_annotation_df.head()

,#id,type,x,y,z,r,pid
0,1,3,3381.6,2941.6,4420,1,2
1,2,3,3382.59,2941.48,4420,1,3
2,3,3,3383.58,2941.35,4420,1,4
3,4,3,3384.1,2941.21,4420.76,1,5
4,5,3,3384.53,2941.07,4421.65,1,6


In [6]:
ds_x= pd.to_numeric(resampled_annotation_df['x'])
ds_x=ds_x/ratioxyz
ds_xround=ds_x.astype(int).astype(str)
ds_y= pd.to_numeric(resampled_annotation_df['y'])
ds_y=ds_y/ratioxyz
ds_yround=ds_y.astype(int).astype(str)

ds_z= pd.to_numeric(resampled_annotation_df['z'])
ds_z=ds_z/ratioxyz
ds_zround=ds_z.astype(int).astype(str)

# rounds by astype int, then conver to string

In [7]:
ds_coordinates= pd.DataFrame(columns=['x','y','z'])
ds_coordinates['x']=ds_xround
ds_coordinates['y']=ds_yround
ds_coordinates['z']=ds_zround
# put in to panda dataframe just in case for future use

In [8]:
q = [' '.join(x) for x in zip(ds_xround,ds_yround,ds_zround)]
# create a list where row1= x1 y1 z1 row2= x2 y2 z2 and etc 
# for writing to the text file

In [23]:
if 'D' in resample_file:
    out_name= outdir[3:]+ f'D_{goal_xyz}voxel_trace_1umStepsize.txt'
else:
    out_name= outdir[3:]+ f'_{goal_xyz}voxel_trace_1umStepsize.txt'
print(out_name)

AL126_25.0voxel_trace_1umStepsize.txt


In [10]:
num_row=len(resampled_annotation_df.index)
f=open(outdir+'/'+out_name,'w+')
f.write('point'+'\n')
f.write(str(num_row)+'\n')

for lines in q:
    f.write(lines+'\n')

f.close()

#works, yay

In [150]:
#attempt to find endings for each branch
# the #n of the ending should not be a parent for something else

#first put these 2 columns in to list
list_n= resampled_annotation_df['#id'].tolist()
list_parent= resampled_annotation_df['pid'].tolist()

##DONOT RUN, some how this takes VERY long. Consider finding the endings using the non-resampled file where we have less points
ending_list=[counter for counter,idx in enumerate(list_n) if idx not in list_parent]


KeyboardInterrupt: 

In [148]:
endings_df=pd.DataFrame(resampled_annotation_df.iloc[ending_list])
endings_df.head()

,#n,type,x,y,z,radius,parent,seg_id,level,mode,timestamp,TFresindex
0,104067,3,4811.632,2360.216,3550.490,1.000,104068,0,0,0,0,0
55,104123,3,4953.056,2357.856,3579.955,1.000,104124,0,0,0,0,0
57,104079,3,4954.408,2358.768,3580.555,1.000,104080,0,0,0,0,0
74,104125,3,4999.888,2360.488,3590.920,1.000,104126,0,0,0,0,0
89,104159,3,5095.344,2369.136,3610.450,1.000,104160,0,0,0,0,0


In [120]:
f=open(outdir+'/endings.txt','w+')

for idx in ending_list:
    loc=q[idx]
    f.write(loc+'\n')
f.close()
    

In [43]:
annotation_df=pd.DataFrame(annotations, columns=headings)
annotation_df.columns

AttributeError: 'str' object has no attribute 'type'